In [0]:
from pyspark.sql.types import StructType, StructField, StringType , IntegerType
schema = StructType([
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone_number", StringType(), True)
])

# Sample data
data = [
    ("Alice", "alice@example.com", "123-456-7890"),
    ("Bob", "bob@example.com", "987-654-3210"),
    ("Charlie", "charlie@example.com", "555-123-4567")
]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show the DataFrame
df.show()

+-------+-------------------+------------+
|   name|              email|phone_number|
+-------+-------------------+------------+
|  Alice|  alice@example.com|123-456-7890|
|    Bob|    bob@example.com|987-654-3210|
|Charlie|charlie@example.com|555-123-4567|
+-------+-------------------+------------+



In [0]:
def masked_email(email):
    split_data = email.split('@')
    username = split_data[0]
    masked_data = "*" * (len(username) - 2) + username[-2:]
    return masked_data + "@" + split_data[1]

In [0]:
def mask_phonenumber(phonenumber):
    split_data=phonenumber.split("-")
    mask_array=split_data[1]
    masked_data="*"*(len(mask_array))
    return split_data[0]+"-"+masked_data+"-"+split_data[2]

In [0]:
masked_email_udf = udf(masked_email, StringType())
masked_mobile_udf= udf(mask_phonenumber, StringType())

In [0]:
from pyspark.sql.functions import *
maskedEmail_df = df.withColumn("masked_email", when(df.email.isNotNull(), masked_email_udf(df.email)).otherwise(None))
masked_mobile=maskedEmail_df.withColumn("masked_phone",  masked_mobile_udf(df.phone_number))
masked_mobile.show()


+-------+-------------------+------------+-------------------+------------+
|   name|              email|phone_number|       masked_email|masked_phone|
+-------+-------------------+------------+-------------------+------------+
|  Alice|  alice@example.com|123-456-7890|  ***ce@example.com|123-***-7890|
|    Bob|    bob@example.com|987-654-3210|    *ob@example.com|987-***-3210|
|Charlie|charlie@example.com|555-123-4567|*****ie@example.com|555-***-4567|
+-------+-------------------+------------+-------------------+------------+

